In [4]:
import spacy
import csv
import pandas as pd
import re
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from collections import OrderedDict
from textblob import TextBlob

def getGrammar(rule,start):
    terminal=[]
    nonterminal=[]    
    r=[]
    k=rule.keys()
    for i in rule.values():
        for j in i:
            for a in list(str(j).split(' ')):
                if a not in k:
                    if a not in r:
                        r.append(a)
    
    r.append("$")
    t=dict()
    for i in rule.keys():
        t[i]=dict()
        for j in r:
            t[i].update({j:set()})
    
    return rule,start,r,t

def Calculate_First(s,rule,v):
    s=list(str(s))
    if s[0] in v:
        return set([s[0]])
    elif s[0] == '`':
        return set(["`"])
    else:
        res = set()
        for j in rule[s[0]]:
            h = Calculate_First(list(str(j).split(' ')),rule,v)
            res.update(set(h))
        if len(s) == 1:
            return res
        else:
            if '`' in res:
                res.remove('`')
                return res.union(Calculate_First(list(s.split(' '))[1:],rule,v))
            return res

        
def getValue(v,rule):
    for key, value in rule.items():
        if value == v:
            return key
        

def Calculate_Follow(s,rule,v,start):
    res = set()
    if s == start:
        res = set(['$'])
    for i in rule.values():
        for j in range(len(i)):
            l = list(i[j].split(' '))
            for k in range(len(l)):
                if l[k] == s:
                    if k == len(l) - 1:
                        if getValue(i,rule) == s:
                            continue
                        else:
                            res.update(set(Calculate_Follow(getValue(i,rule),rule,v,start)))
                    else:
                        c = set(Calculate_First(l[k + 1:],rule,v))
                        if '`' in c:
                            c.remove('`')
                            fol = Calculate_Follow(getValue(i,rule),rule,v,start)
                            c.update(fol)
                            res.update(c)
                        else:
                            res.update(set(c))
    return res


def parseString(string,t,start):
    string.append('$')
    stk = ['$', start]
    while not len(stk) == 0:
        top = stk[-1]
        stk.pop()
        if string[0] == '$' and top == '$' and len(stk) == 0:
            return 0
        elif (string[0] == '$' and len(stk) != 0) or (string[0] != '$' and len(stk) == 0):
            return 1
        elif top == string[0]:
            string = string[1:]
        else:
#             for i in t[top][string[0]]:
#                 l = list(i.split(' '))
            for j in string[::-1]:
                stk.append(j)
    

nlp = spacy.load("en_core_web_sm")
s=input("Enter a string you want to translate : ")
s=s.lower()

res = re.sub(r'[^\w\s]', '', s)
res=res.strip()

str1 = list(res.split(" "))
str2=res

doc = nlp(res)
a=[]
b=[]
for token in doc:
    b.append([token,token.pos_])
    a.append((token,token.pos_))


words, tags = zip(*a)


d1={}
for i in range(len(tags)):
    e=[]
    for j in range(len(b)):
        if tags[i]==b[j][1]:
            e.append(b[j][0])
    d1.update({tags[i]:list(set(e))})



ke = []
for key in d1.keys():
    ke.append(key)

d11 = {"N":[], "P":[] ,"V": [],"ADVERB": [],"PROPN":[],"ADJ":[],"ADP":[],"D":[],"CONJ":[],"INTJ":[],"PN":[]}

rules = {"S": ["NP VP"], "NP": ["P", "PN", "D N"], "VP": ["V NP"], "N":[],"P":[],"D":[],"V":[],"PN":[]}

if "AUX" in d1.keys():
    rules["V"]+=d1["AUX"]
    d1.pop("AUX")
if "CCONJ" in d1.keys():
    rules["CONJ"]+=d1["CCONJ"]
    d1.pop("CCONJ")
if "SCONJ" in d1.keys():
    rules["CONJ"]+=d1["SCONJ"]
    d1.pop("SCONJ")

for k,v in d1.items():
    if k=="NOUN":
        rules["N"]=d1[k]
    if k=="PRON":
        rules["P"]=d1[k]
    if k=="PROPN":
        rules["PN"]=d1[k]
    if k=="DET":
        rules["D"]=d1[k]

d1=d11


res=s.split()

rules.update(d1)


start="S"

d,start,r,t=getGrammar(rules,start)

print()

for i in d.keys():
    for rule in d[i]:
        fir = Calculate_First(rule,d,r)
        if '`' in fir:
            fol = Calculate_Follow(i,d,r,start)
            for j in fol:
                if j != '$':
                    t[i][j].add(rule)
            if '$' in fol:
                t[i]['$'].add(rule)
        else:
            for j in fir:
                t[i][j].add(rule)

flagg = parseString(res,t,start)

if flagg==0:
    keys = []
    
    ans=""
    
    dict11={"is":"है","a":"एक","they":"वे","them":"उन्हें","running":"दौंड़ रहा","sleeping":"सो रहा",
            "eating":"खा रहा","win":"जीत","lose":"हार"}
    
    for key in dict11.keys():
        keys.append(key)
        
    for i in range(len(str1)):
        if str1[i] in keys:
            ans=ans+dict11[str1[i]]+" "
        else:
#             ans=ans+transliterate(i, sanscript.ITRANS, sanscript.DEVANAGARI)+" "
            ttt=str1[i]
            blob = TextBlob(ttt)
            ans+=str(blob.translate(from_lang='en',to='hi'))+" "
    
    print("Translated String : ",end=" ")
    print(ans)
else:
    print("NOT POSSIBLE")

# John is a boy

Enter a string you want to translate : eating is John

NOT POSSIBLE
